In [45]:
!pip install hmmlearn
!pip install scikit-learn

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from hmmlearn import hmm

In [ ]:
##reading from the text file for observation sequnce. Only using 4.0 for now

def readInputFromFile(score):
    file_path = score
    with open(file_path, "r") as file:
        lines = file.readlines()
        seqs = lines[0].split("K")
        words = " ".join(seqs).split(" ")
        string_id = {word: i for i, word in enumerate(set(words))}

        seqs = [s.split(" ") for s in seqs]
        ##convert words in seqs to numbers
        seqs = [[string_id[w] for w in s] for s in seqs]
        # lines = file.readlines()
        # print(len(lines))
        # string_to_id = {string: i for i, string in enumerate(set(lines[0]))}
        # raw_input = lines[0].split("K")
        # print(len(raw_input))
        # raw_string_pre_pad = [s.split(" ") for s in raw_input ]
        
        max_seq = max(len(s) for s in seqs)
        hmm_input = [np.pad(s, (0, max_seq - len(s)), constant_values=9999) for s in seqs]
        # print(hmm_input[0])
    return hmm_input

train_input = readInputFromFile("train/3.5.txt")
test_input = readInputFromFile("test/3.5.txt")

In [ ]:
# with open("train/3.5.txt", "r", encoding='utf-8') as file:
#     lines = file.readlines()
#     seqs = lines[0].split("K")
#     words = " ".join(seqs).split(" ")
#     string_id = {word: i for i, word in enumerate(set(words))}

#     seqs = [s.split(" ") for s in seqs]
#     print(seqs[0])
#     seqs = [[string_id[w] for w in s] for s in seqs]
#     print(seqs[0])
            
#     print(string_id)
#     max_seq = max(len(s) for s in seqs)
#     hmm_input = [np.pad(s, (0, max_seq - len(s)), constant_values=9999) for s in seqs]
#     print(hmm_input[0])



In [31]:
best_score = best_model = None
n_fits = 20
N = 10
np.random.seed(13)

for idx in range(n_fits):
    model = hmm.CategoricalHMM(
        n_components=N, random_state=idx,
        init_params='se')  # don't init transition, set it below
    # we need to initialize with random transition matrix probabilities
    # because the default is an even likelihood transition
    # we know transitions are rare (otherwise the casino would get caught!)
    # so let's have an Dirichlet random prior with an alpha value of
    # (0.1, 0.9) to enforce our assumption transitions happen roughly 10%
    # of the time

    ##A 
    epsilon = 1e-6
    transmat = np.full((N, N), 1 / N) + epsilon
    transmat /= transmat.sum(axis=1, keepdims=True)
    # print(A_matrix)
    # model.transmat_ = np.array([np.random.dirichlet([0.9, 0.1]),
    #                             np.random.dirichlet([0.1, 0.9])])
    model.transmat_ = transmat
    model.fit(train_input)
    score = model.score(test_input)
    print(f'Model #{idx}\tScore: {score}')
    if best_score is None or score > best_score:
        best_model = model
        best_score = score

Model #0	Score: -1714932.8416720838
Model #1	Score: -1739949.7450997666
Model #2	Score: -1580008.1390205447
Model #3	Score: -1810791.3952595687
Model #4	Score: -2444928.006717306
Model #5	Score: -1700623.433868408
Model #6	Score: -1692989.7323168127
Model #7	Score: -1834652.05370565
Model #8	Score: -1756916.4325049762
Model #9	Score: -1860037.1649686703
Model #10	Score: -2102721.1316342535
Model #11	Score: -1822475.1106707456
Model #12	Score: -1525690.3733029861
Model #13	Score: -2033096.578521973
Model #14	Score: -1605001.577602776
Model #15	Score: -1869345.2464626175
Model #16	Score: -2354373.0392343053
Model #17	Score: -1594687.945320199
Model #18	Score: -1787100.62473192
Model #19	Score: -1580193.6677048628
Model #20	Score: -2031373.7301996883


KeyboardInterrupt: 

In [40]:
import pickle
with open("model" + str(best_score) + ".pkl", "wb") as file: 
    pickle.dump(best_model, file)

In [41]:
print("model" + str(best_score) + ".pkl")
with open("model" + str(best_score) + ".pkl", "rb") as file: 
    read_best_model = pickle.load(file)

model-1525690.3733029861.pkl


In [43]:
read_best_model.predict(test_input)

array([1, 1, 4, ..., 0, 0, 0])